Welcome, in this notebook I will study the Capstone project included in the IBM Data science professional certificate

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# WEB SCRAPING

I am going to scrape the available tab on https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
For this I am simply going to retrieve the HTML page using pandas. By doing so it contains the table I want to retrieve.

In [3]:
liste = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(liste[0]) #Getting the table only

In [4]:
#df #Let's take a look


In [5]:
import re
new_df = pd.DataFrame()
dict_list = []
# Let's browse the table to make the required dataframe.
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        string = df.loc[i,j]
        postal_code = string[:3]
        if string[3:] != 'Not assigned': #Ignoring the non assigned postal codesd
            try:
                borough = string[3:].split('(')[0]
                found = string[3:].split('(')[1]
                found = found[found.find("(")+1:found.find(")")]
                neighborhood = found.replace('/',',')
                neighborhood = neighborhood.split(',')[0]
                borough = borough.replace('/',',')
                dictio = {'Postal code': postal_code, 'Borough':borough, 'Neighborhood': neighborhood}
                dict_list.append(dictio)
            except:
                print("No neighborhood " + str(string[3:]))
                borough = string[3:].replace('/',',')
                neighborhood = borough
                neighborhood = neighborhood.split(',')[0]
                dictio = {'Postal code': postal_code, 'Borough':borough, 'Neighborhood': neighborhood}
                dict_list.append(dictio)
new_df = pd.DataFrame.from_dict(dict_list)

No neighborhood Queen's Park / Ontario Provincial Government


In [6]:
#for row in new_df['Neighborhood']:
 #   print(row)

In [7]:
new_df.shape

(103, 3)

In [8]:
dfll = pd.read_csv('Geospatial_Coordinates.csv')
dfll

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
df = new_df
df['Latitude'] = dfll['Latitude']
df['Longitude'] = dfll['Longitude']
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,Regent Park,43.763573,-79.188711
3,M6A,North York,Lawrence Manor,43.770992,-79.216917
4,M7A,"Queen's Park , Ontario Provincial Government",Queen's Park,43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway,43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,Old Mill South,43.739416,-79.588437


In [10]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Canada'

from geopy.exc import GeocoderTimedOut

def do_geocode(address,geolocator):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address,geolocator)

geolocator = Nominatim(user_agent="canada_explorer")

#location = do_geocode(address,geolocator)
#Always get a time out error, got the coordinates manually 
#latitude = location.latitude
#longitude = location.longitude
latitude = 43.715062
longitude = -79.372571

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Canada using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

The geograpical coordinate of Toronto are 43.715062, -79.372571.


# Clustering

Now that we have displayed the neighborhoods on a map we can see how they cluster together thanks to the Foursquare data.

## 

In [11]:
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

CLIENT_ID = 'SJ5ILVLRNBXCWC30C1WKSNCLH2JQVDW451GEMOJ4S4Z3NWLI' # your Foursquare ID
CLIENT_SECRET = 'YFGB1YKWXORVA3TUOU3O1RN5TRMZDVHD5L54BMXHQMKYHE1E' # your Foursquare Secret
VERSION = '20200301'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

canada_data = df

neighborhood_latitude = canada_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = canada_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = canada_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# type your answer here
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

Your credentails:
CLIENT_ID: SJ5ILVLRNBXCWC30C1WKSNCLH2JQVDW451GEMOJ4S4Z3NWLI
CLIENT_SECRET:YFGB1YKWXORVA3TUOU3O1RN5TRMZDVHD5L54BMXHQMKYHE1E
Latitude and longitude values of Parkwoods are 43.806686299999996, -79.19435340000001.
2 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here

canada_venues = getNearbyVenues(names=canada_data['Neighborhood'],
                                   latitudes=canada_data['Latitude'],
                                   longitudes=canada_data['Longitude']
                                  )
print(canada_venues.shape)
canada_venues.head()

canada_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(canada_venues['Venue Category'].unique())))

(2241, 7)
There are 268 uniques categories.


In [13]:
# one hot encoding
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]

canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')
    
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Café,Breakfast Spot,Furniture / Home Store,Bar,Intersection,Italian Restaurant,Japanese Restaurant,Climbing Gym,Restaurant
1,Bathurst Manor,Coffee Shop,Bank,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Restaurant,Diner,Ice Cream Shop,Supermarket,Sushi Restaurant
2,Bayview Village,Indian Restaurant,Yoga Studio,Coffee Shop,Sandwich Place,Burger Joint,Bus Line,Restaurant,Pizza Place,Pharmacy,Discount Store
3,Bedford Park,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Cosmetics Shop,Beer Bar,Breakfast Spot,Bakery,Diner
4,Berczy Park,Cafeteria,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


# Cluster Neighborhoods

In [14]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

canada_grouped_clustering

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0
2,0.047619,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.047619,0.00,0.000000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.0,0.0,0.01,0.000000,0.0,0.000000,0.000000,0.01,0.000000,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.022222,0.0,0.011111,0.0,0.0,0.0,0.0,0.0,0.0,0.011111,...,0.0,0.0,0.00,0.000000,0.0,0.011111,0.000000,0.00,0.011111,0.0
89,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.030303,0.000000,0.00,0.000000,0.0
90,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0
91,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0


In [15]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0])

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = canada_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

canada_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
1,M4A,North York,Victoria Village,43.784535,-79.160497,0.0,Moving Target,Bar,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Diner
2,M5A,Downtown Toronto,Regent Park,43.763573,-79.188711,0.0,Mexican Restaurant,Intersection,Breakfast Spot,Electronics Store,Spa,Bank,Rental Car Location,Medical Center,Moving Target,Discount Store
3,M6A,North York,Lawrence Manor,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
4,M7A,"Queen's Park , Ontario Provincial Government",Queen's Park,43.773136,-79.239476,0.0,Hakka Restaurant,Gas Station,Bank,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Bakery,Drugstore,Donut Shop


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

In [18]:
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Neighborhood'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.7).add_to(map_clusters)
        
        
map_clusters